In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import os
import pandas as pd
import cv2
import torch.optim as optim
from tqdm import tqdm
import numpy as np

## preprocessing

In [20]:
train_images = []
for i in range(1, 50000 + 1):
    if i%10000==0:
        print(i//10000)
    im = cv2.imread(os.path.join('train', f"{i}.png"), cv2.IMREAD_GRAYSCALE)
    if im is not None:
        
        if np.random.rand() > 0.5:
            im = cv2.flip(im, 1)
        
        im = torch.tensor(im, dtype=torch.float32).unsqueeze(0)  # [1, 32, 32]
        im = im / 255.0  # Normalization
        train_images.append(im)

train_images = torch.stack(train_images)

1
2
3
4
5


In [21]:
labels = pd.read_csv("trainLabels.csv")
labels['numeric_labels'] = pd.factorize(labels['label'])[0]

main_labels=torch.tensor(labels['numeric_labels'])

print(main_labels[:5])

tensor([0, 1, 1, 2, 3])


In [22]:
print(main_labels[:5])
print(train_images.shape)

tensor([0, 1, 1, 2, 3])
torch.Size([50000, 1, 32, 32])


In [23]:
model = nn.Sequential(
    nn.Conv2d(1, 16, 3, padding=1),  # [batch, 16, 32, 32]
    nn.ReLU(),
    nn.BatchNorm2d(16),  #  BN layer (16 channels)
    nn.MaxPool2d(2, 2),  # [batch, 16, 16, 16]
    nn.Conv2d(16, 32, 3, padding=1),  # [batch, 32, 16, 16]
    nn.ReLU(),
    nn.BatchNorm2d(32),  #  BN layer (32 channels)
    nn.MaxPool2d(2, 2),  # [batch, 32, 8, 8]
    nn.Flatten(),
    nn.Linear(32 * 8 * 8, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),  
    nn.Linear(128, 10)
)

In [24]:
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Better optimizer than SGD lol
criterion = nn.CrossEntropyLoss()

In [28]:
batch_size = 64
num_epochs = 5 # total 15 epochs with theese 5. 

for epoch in range(num_epochs):
    epoch_loss = 0
    correct = 0
    for i in range(0, len(train_images), batch_size):
        batch_images = train_images[i:i+batch_size]
        # 
        batch_labels = main_labels[i:i+batch_size]
        
        optimizer.zero_grad()
        outputs = model(batch_images)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == batch_labels).sum().item()
    
    accuracy = 100 * correct / len(train_images)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss/len(train_images):.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1, Loss: 0.0017, Accuracy: 96.78%
Epoch 2, Loss: 0.0009, Accuracy: 98.90%
Epoch 3, Loss: 0.0007, Accuracy: 99.13%
Epoch 4, Loss: 0.0015, Accuracy: 96.74%
Epoch 5, Loss: 0.0016, Accuracy: 96.36%
